In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
print(cv2.__version__)

4.10.0


In [2]:
pip install opencv-contrib-python

In [1]:
import cv2
import numpy as np
from collections import deque

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Dummy gender classification function (replace with real model if available)
def classify_gender(face_img):
    return np.random.choice(["Male", "Female"])

# Check if a woman is surrounded by men
def is_surrounded(woman_center, men_centers, min_distance=100):
    for man_center in men_centers:
        if np.linalg.norm(np.array(woman_center) - np.array(man_center)) < min_distance:
            return True
    return False

# Moving average function to smooth positions
def moving_average(positions, window_size=5):
    positions_list = list(positions)  # Convert deque to list
    if len(positions_list) < window_size:
        return np.mean(positions_list, axis=0)
    return np.mean(positions_list[-window_size:], axis=0)

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Store positions for smoothing
women_positions = deque(maxlen=10)

print("Press 'q' to quit.")
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Resize for consistent processing
    frame = cv2.resize(frame, (640, 480))

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Det ect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    women_centers = []
    men_centers = []

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        if face_img.size > 0:
            # Predict gender for the detected face
            gender = classify_gender(face_img)
            center = (x + w // 2, y + h // 2)

            # Assign colors and categorize centers
            if gender == "Female":
                women_centers.append(center)
                women_positions.append(center)
                color = (255, 0, 0)  # Blue for women
            else:
                men_centers.append(center)
                color = (0, 255, 0)  # Green for men

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Check for women surrounded by men
    for woman_center in women_centers:
        smoothed_center = moving_average(women_positions, window_size=5)
        if is_surrounded(woman_center, men_centers):
            cv2.putText(frame, "Surrounded by men", (int(smoothed_center[0]) - 50, int(smoothed_center[1]) - 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Lone woman", (int(smoothed_center[0]) - 50, int(smoothed_center[1]) - 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Display the processed frame
    cv2.imshow('Gender Detection', frame)

    # Break on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Press 'q' to quit.
